In word embeddings, every word is represented as an n-dimensional dense vector. The words that are similar will have similar vector. Word embeddings techniques such as GloVe and Word2Vec have proven to be extremely efficient for converting words into corresponding dense vectors.

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [ ]:
# parameters are vocabulary or total number of unique words in corpus, dimensions for each word vector, length of the input sentence
embedding_layer = Embedding(200, 32, input_length=50)

#Custom Word Embedding

In [ ]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

The output of the word embedding is a 2D vector where words are represented in rows, whereas their corresponding dimensions are presented in columns. Finally, if you wish to directly connect your word embedding layer with a densely connected layer, you first have to flatten your 2D word embeddings into 1D. 

In [ ]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [ ]:
unique_words = set(all_words)
print(len(unique_words))

45


In [ ]:
vocab_length = 50

The Embedding layer expects the words to be in numeric form. Therefore, we need to convert the sentences in our corpus to numbers. One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [ ]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[7, 45, 12, 19, 47], [22, 31, 47, 49, 2, 21, 14], [7, 42, 1, 14, 45, 23], [48, 6], [46, 32, 27, 22, 2, 21, 14], [25, 49, 49, 17], [29, 34, 32, 49, 23, 47], [14, 45, 49, 43, 47], [12, 8], [9, 3, 30], [13, 9], [14, 47, 22, 27], [2, 13, 18, 21, 22, 47], [22, 47, 47, 12], [2, 6, 18, 17], [22, 8, 45, 13]]


The embedding layer expects sentences to be of equal size. However, our encoded sentences are of different sizes. One way to make all the sentences of uniform size is to increase the lenght of all the sentences and make it equal to the length of the largest sentence. Let's first find the largest sentence in our corpus and then we will increase the length of all the sentences to the length of the largest sentence.

In [ ]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

we use a lambda expression to find the length of all the sentences. We then use the max function to return the longest sentence. Finally the longest sentence is tokenized into words and the number of words are counted using the len function.
To make all the sentences of equal size, we will add zeros to the empty indexes that will be created as a result of increasing the sentence length. To append the zeros at the end of the sentencses, we can use the pad_sequences method. 

In [ ]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[ 7 45 12 19 47  0  0]
 [22 31 47 49  2 21 14]
 [ 7 42  1 14 45 23  0]
 [48  6  0  0  0  0  0]
 [46 32 27 22  2 21 14]
 [25 49 49 17  0  0  0]
 [29 34 32 49 23 47  0]
 [14 45 49 43 47  0  0]
 [12  8  0  0  0  0  0]
 [ 9  3 30  0  0  0  0]
 [13  9  0  0  0  0  0]
 [14 47 22 27  0  0  0]
 [ 2 13 18 21 22 47  0]
 [22 47 47 12  0  0  0]
 [ 2  6 18 17  0  0  0]
 [22  8 45 13  0  0  0]]


We will create a very simple text classification model with an embedding layer and no hidden layers. 
we create a Sequential model and add the Embedding layer as the first layer to the model. The length of the vocabulary is specified by the vocab_length parameter. The dimension of each word vector will be 20 and the input_length will be the length of the longest sentence, which is 7. Next, the Embedding layer is flattened so that it can be directly used with the densely connected layer. Since it is a binary classification problem, we use the sigmoid function as the loss function at the dense layer.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 7, 20)             1000      
                                                                 
 flatten_2 (Flatten)         (None, 140)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 141       
                                                                 
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


You can see that the first layer has 1000 trainable parameters. This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuran.

In [ ]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 408ms/step - loss: 0.6859 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6822 - acc: 0.6250
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6784 - acc: 0.6250
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6747 - acc: 0.6875
Epoch 5/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6709 - acc: 0.6875
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6672 - acc: 0.6875
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6634 - acc: 0.8750
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.6597 - acc: 0.8750
Epoch 9/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6559 - acc: 0.8750
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6521 - acc: 0.9375
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6

In [ ]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


#Loading Pretrained Word Embeddings

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

In [ ]:
from google.colab import files
files.upload()

Saving glove.6B.100d.txt to glove.6B.100d.txt


In [ ]:
embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
# from keras.layers.embeddings import Embedding

In [ ]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [ ]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from keras.preprocessing.text import Tokenizer

we used one_hot function to convert text to vectors. Another approach is to use Tokenizer function from keras.preprocessing.text library.

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)


To get the number of unique words in the text, you can simply count the length of word_index dictionary of the word_tokenizer object. add 1 with the vocabulary size. This is to store the dimensions for the words for which no pretrained word embeddings exist.

In [ ]:
vocab_length = len(word_tokenizer.word_index) + 1

to convert sentences to their numeric counterpart, call the texts_to_sequences function and pass it the whole corpus.

In [ ]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


In [ ]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


load the GloVe word embeddings and then create our embedding matrix that contains the words in our corpus and their corresponding values from GloVe embeddings.

We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values, in the form of an array.

In [ ]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

The dictionary embeddings_dictionary now contains words and corresponding GloVe embeddings for all the words. 

word_tokenizer.word_index dictionary that contains our words and their corresponding index. 

Each word will be passed as key to the embedding_dictionary to retrieve the corresponding 100 dimensional vector for the word. The 100 dimensional vector will then be stored at the corresponding index of the word in the embedding_matrix

In [ ]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

embedding_matrix now contains pretrained word embeddings for the words in the corpus

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 7, 100)            4400      
                                                                 
 flatten_1 (Flatten)         (None, 700)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 701       
                                                                 
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


since we have 44 words in our vocabulary and each word will be represented as a 100 dimensional vector, the number of parameters for the embedding layer will be 44 x 100 = 4400. The output from the embedding layer will be a 2D vector with 7 rows (1 for each word in the sentence) and 100 columns. The output from the embedding layer will be flattened so that it can be used with the dense layer.

In [ ]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 441ms/step - loss: 0.7247 - acc: 0.6250
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6993 - acc: 0.6875
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6749 - acc: 0.6875
Epoch 4/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6515 - acc: 0.6875
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6291 - acc: 0.6875
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6075 - acc: 0.6875
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5867 - acc: 0.6875
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5666 - acc: 0.6875
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5473 - acc: 0.8125
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.5287 - acc: 0.8125
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 0.5

In [ ]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000
